In [2]:
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt



In [ ]:
class Actions:
    def __init__(self, actions):
        self.all_actions = actions
        self.history = [np.random.randint(len(actions))]
    
    def next(self, action):
        self.history.append(action)

    def last(self):
        return self.history[-1]

class Costs:
    def __init__(self):
        self.history = []
        self.accumulated = 0

    def receive(self, cost):
        self.accumulated += cost
        self.history.append(cost)

    def last(self):
        return self.history[-1]

class Brain:
    def __init__(self, input_size=2, hidden_size=4, num_layers=2):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        # self.hidden = torch.zeros(num_layers, 1, hidden_size)

        self.optimizer = torch.optim.Adam(self.rnn.parameters(), lr=0.01)
        self.loss_fn = nn.NLLLoss()

    def train(self, action, cost):
        self.optimizer.zero_grad()
        self.loss_fn(action, cost).backward()

brain = Brain()

class Player:
    def __init__(self, actions):
        self.actions = Actions(actions)
        self.costs = Costs()

        self.brain = brain
        self.h = torch.randn(brain.num_layers, 1, brain.hidden_size)

    def take_action(self, state):
        self.out, self.h = self.brain.rnn(torch.as_tensor(state, dtype=torch.float32).view(1, 1, brain.input_size), self.h)
        action = self.out.argmax().item()
        self.actions.next(action)
        return action

    def receive_cost(self, cost, train=True):
        self.costs.receive(cost)
        # use the cost to train the brain rnn
        




In [34]:
nn.NLLLoss()(torch.as_tensor([0.1, 0.2, 0.3, 0.4], dtype=torch.float32).view(1, 4), torch.as_tensor([2], dtype=torch.long))

tensor(-0.3000)

In [29]:
brain = nn.RNN(input_size=2, hidden_size=4, num_layers=2)
x = torch.randn(1, 1, 2)
h0 = torch.randn(2, 1, 4)
output, hn = brain(x, h0)
print(output)

tensor([[[ 0.5896,  0.4205, -0.1433, -0.5489]]], grad_fn=<StackBackward0>)


In [ ]:
class CongestionGame:
    def __init__(self, n_players=60, cost_setup=(15, 2, 0.2, 22.96)):
        self.cost_f, self.cost_a, self.cost_a_mult, self.cost_best = cost_setup
        self.actions = ['fa', 'fbf', 'af', 'aba']
        
